In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [2]:
batch_size = 64
epochs = 30
latent_dim = 256
num_samples = 10000
data_path = 'data/fra-eng/fra.txt'

In [3]:
# Собираем из текстов токены

input_texts = []
target_texts = []

input_vocab = set()
output_vocab = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    for word in input_text.split():
        input_vocab.add(word.strip())
    target_texts.append(target_text)
    for word in target_text.split():
        output_vocab.add(word.strip())
    
input_vocab2index = {word: i+2 for i, word in enumerate(input_vocab)}
output_vocab2index = {word: i+2 for i, word in enumerate(output_vocab)}

In [4]:
def indexesFromSentence(sentence, vocab):
    return [vocab[word.strip()] for word in sentence.split(' ')]

def tensorFromSentence(sentence, vocab):
    indexes = indexesFromSentence(sentence, vocab)
    indexes.append(1)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)

def tensorsFromSent(input_sentences, output_sentences):
    input_tensor = tensorFromSentence(input_sentences, input_vocab2index)
    target_tensor = tensorFromSentence(output_sentences, output_vocab2index)
    return (input_tensor, target_tensor)

In [5]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def call_attention(self, hidden, encoder_outputs):
        attn_weights = F.softmax(torch.bmm(hidden, encoder_outputs.T.unsqueeze(0)),dim=-1)
        attn_output = torch.bmm(attn_weights, encoder_outputs.unsqueeze(0))
        concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        return concat_output

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        #output = torch.cat((embedded[0], attn_applied[0]), 1)
        #output = self.attn_combine(output).unsqueeze(0)
        #output = F.relu(output)
        _, hidden = self.gru(embedded, hidden)
        concat_output = self.call_attention(hidden, encoder_outputs)

        output = F.log_softmax(self.out(concat_output), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [7]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[0]])

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == 1:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [13]:
encoder = EncoderRNN(len(input_vocab2index)+2, 30)
attn_decoder1 = AttnDecoderRNN(30, len(output_vocab2index)+2, dropout_p=0.1)

encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = torch.optim.SGD(attn_decoder1.parameters(), lr=0.01)
training_pairs = np.random.randint(0, len(input_texts), size=10)
criterion = nn.NLLLoss()

print_loss_total = 0
for i in range(10):
    input_tensor, target_tensor = tensorsFromSent(input_texts[training_pairs[i]], target_texts[training_pairs[i]])

    loss = train(input_tensor, target_tensor, encoder,
               attn_decoder1, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    
    print_loss_avg = print_loss_total / 1
    print_loss_total = 0
    print('(%d %d%%) %.4f' % (i, i / 10 * 100, print_loss_avg))